In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [3]:
mbti= pd.read_csv('/Users/gonzaloalvarez/final_project/mbti_changed.csv')

In [4]:
mbti.head()

,type,posts,type_index,cleaned_text
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,enfp and intj moments sportscenter not t...
1,ENTP,'I'm finding the lack of me in these posts ver...,3,im finding the lack of me in these pos...
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,good one of course to which i say i ...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,dear intp i enjoyed our conversation the...
4,ENTJ,'You're fired.|||That's another silly misconce...,2,youre fired thats another silly misconcep...


#### Split the dataset in train, validation and test

In [51]:
train,test= train_test_split(mbti)
train, val= train_test_split(train)


print(train.shape)
print(test.shape)
print(val.shape)

(4879, 4)
(2169, 4)
(1627, 4)


#### Word embedding --> Text to arrays

In [52]:
embed= hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
hub_layer=hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train.cleaned_text[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 1.2631984 , -0.07429716, -1.4959615 ,  1.8658408 ,  0.4650373 ,
        -0.9525655 ,  1.0888932 , -0.28460523, -2.4665818 ,  2.6978812 ,
         0.15715301,  0.81745684, -0.03829916, -0.11094519, -1.2923864 ,
        -0.40714106, -1.1507365 ,  0.2363303 ,  1.4926238 , -2.779092  ,
         0.21276763,  0.2932151 ,  3.0795908 , -0.05187522, -1.9826746 ,
         1.386021  , -4.4261975 ,  0.12340773,  0.7434004 , -1.6217937 ,
        -1.2666503 ,  1.9982475 ,  2.9582098 , -1.5144217 , -2.3716185 ,
         0.44746035,  0.8203376 , -1.388145  ,  0.6298591 , -1.7929841 ,
         1.7380686 ,  1.7233695 , -0.74018145,  1.1994958 ,  0.11210676,
        -0.6749734 , -1.6701362 , -1.6129097 ,  1.0539674 ,  0.6284371 ],
       [ 1.6607497 , -0.2924295 , -1.0666468 ,  1.9289665 ,  0.7001575 ,
        -0.5796863 ,  1.0429944 , -0.30806702, -2.3338497 ,  2.226193  ,
         0.14744076,  1.1733816 , -0.25746828,  0.62752044, -1.1856852 ,
 

#### Create the model

In [53]:
model=tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='softmax'))
model.add(tf.keras.layers.Dense(16))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_6 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
Total params: 48,191,688
Trainable params: 48,191,688
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.compile(optimizer='adam', 
             loss= tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=tf.metrics.CategoricalAccuracy())

In [55]:
one_hot_labels = tf.keras.utils.to_categorical(train.type_index.values, num_classes=16)
val_labels=tf.keras.utils.to_categorical(val.type_index.values, num_classes=16)

In [56]:
train_array= np.array(train['cleaned_text'])
val_array= np.array(val['cleaned_text'])

In [68]:
model.fit(train_array, one_hot_labels, verbose=1, validation_data=(val_array, val_labels), epochs=15)

Epoch 1/15
153/153 [==============================] - 86s 559ms/step - loss: 1.3060 - categorical_accuracy: 0.4831 - val_loss: 2.0274 - val_categorical_accuracy: 0.3534
Epoch 2/15
153/153 [==============================] - 84s 549ms/step - loss: 1.2923 - categorical_accuracy: 0.4833 - val_loss: 1.9607 - val_categorical_accuracy: 0.3669
Epoch 3/15
153/153 [==============================] - 84s 547ms/step - loss: 1.2812 - categorical_accuracy: 0.4831 - val_loss: 1.9970 - val_categorical_accuracy: 0.3602
Epoch 4/15
153/153 [==============================] - 83s 544ms/step - loss: 1.2723 - categorical_accuracy: 0.4833 - val_loss: 1.9987 - val_categorical_accuracy: 0.3651
Epoch 5/15
153/153 [==============================] - 83s 540ms/step - loss: 1.2622 - categorical_accuracy: 0.4864 - val_loss: 1.9957 - val_categorical_accuracy: 0.3534
Epoch 6/15
153/153 [==============================] - 82s 539ms/step - loss: 1.2511 - categorical_accuracy: 0.4868 - val_loss: 2.0135 - val_categorical_acc

In [69]:
test_hot_labels = tf.keras.utils.to_categorical(test.type_index.values, num_classes=16)
test_array= np.array(test['cleaned_text'])

In [70]:
results= model.evaluate(test_array, test_hot_labels, verbose=1)


68/68 [==============================] - 0s 6ms/step - loss: 2.0115 - categorical_accuracy: 0.3757


In [71]:
results

[2.011539936065674, 0.3757492005825043]